In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 1.8 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 11.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=87beb3c2084de50a6c64b79314d4490cdf589e47ca61da0cd664106ec4d7faf8
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 13.0 MB/s 
     |████████████████████████████████| 769 kB 12.4 MB/s 
     |████████████████████████████████| 3.0 MB 39.9 MB/s 
     |████████████████████████████████| 895 kB 44.5 MB/s 


In [2]:
!pip install qiskit

     |████████████████████████████████| 6.5 MB 9.7 MB/s 
     |████████████████████████████████| 18.0 MB 97 kB/s 
     |████████████████████████████████| 238 kB 46.6 MB/s 
     |████████████████████████████████| 200 kB 49.4 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 1.6 MB 42.4 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
     |████████████████████████████████| 49 kB 4.3 MB/s 
     |████████████████████████████████| 943 kB 35.0 MB/s 
     |████████████████████████████████| 37.5 MB 23.2 MB/s 
     |████████████████████████████████| 38.1 MB 1.4 MB/s 
     |████████████████████████████████| 3.6 MB 37.9 MB/s 
     |████████████████████████████████| 113 kB 46.8 MB/s 
  Created wheel for qiskit: filename=qiskit-0.35.0-py3-none-any.whl size=11839 sha256=36064059005bd9da7d98f6c7762e1e377c80ddee196703322ca9ade0961bad06
  Stored in directory: /root/.cache/pip/wheels/1e/0a/42/93e5cc11795fb0a85fcf6686eac2791def0c9a1cad0a77

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-yvpq2g0m
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-yvpq2g0m
     |████████████████████████████████| 132 kB 11.7 MB/s 
     |████████████████████████████████| 4.5 MB 48.2 MB/s 
     |████████████████████████████████| 3.8 MB 46.4 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 6.5 MB 41.4 MB/s 
     |████████████████████████████████| 596 kB 40.8 MB/s 
     |████████████████████████████████| 79 kB 6.6 MB/s 
     |████████████████████████████████| 8.6 MB 31.8 MB/s 
     |████████████████████████████████| 127 kB 43.5 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=5c2a9cb5fffed3a25a3387568d42af92b341c69a5e53c0576b51e9654db64de3
  Stored in directory: /tmp/pip-ephem-wheel-cache-gon6p60e/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd0

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [7]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
# pytorch 에서 cpu 사용 선택
# cpu 환경에서 모델 구동시
device = torch.device("cuda:0")

In [9]:
# 사전 훈련된 kobert model 불러옴
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [10]:
# 학습에 사용할 데이터 셋 클래스 선언
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [11]:
# 모델 파라미터 설정
# 토큰의 최대 길이라고 생각
max_len = 64
# 몇 개의 샘플들을 예측해보고 가중치를 업데이트 할 지 설정
# 아래와 같이 배치 사이즈가 64인 경우 데이터 64개 마다 예측한 것을 실제 값과 비교한다
batch_size = 64
warmup_ratio = 0.1
# epoch 횟수는 모델이 전체 데이터셋을 훈련시킬 횟수를 의미한다.
num_epochs = 10
max_grad_norm = 1
log_interval = 200
# learning_rate 값이 너무 크면 원하는 값에 도달하기 힘들고, 너무 작으면 학습기간이 오래 걸린다.
learning_rate =  5e-5

In [12]:
# 분류에 사용할 모델 클래스 선언
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 # num_classes는 카테고리의 개수를 의미한다. (현재 데이터셋의 경우 3개의 분류로 데이터셋이 구성되어 있음)
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [13]:
# to(Device)는 위의 torch.device("cuda:0")를 의미하며 
# GPU에서 학습된 모델을 GPU로 불러올 때 사용한다.
# 또한 GPU로 학습된 모델에 데이터를 제공할 때도 to(Device)를 붙여줘야 함.
# bert모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [14]:
# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [15]:
# optimizer = 전체 데이터 셋의 실제 결과와 모델이 예측한 값 간의 차이가 효율적으로 좁혀질 수 있도록 최적화해주는 역할
# transform에서 제공하는 AdamW optimizer 사용
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# 예측값과 실제값의 오차인 손실함수는 pytorch에서 제공하는 다중분류를 위한 대표적인 손실함수인 torch.nn.CrossEntropyLoss 사용
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [16]:
# 전체 모델 불러오기(cpu 환경으로 사용할 때)
sentimentModel = torch.load('/content/drive/MyDrive/Colab Notebooks/jejuRecommand/jeju_model.pt')

# dict 불러오기
# model1.load_state_dict(torch.load('7emotions_model_state_dict.pt'))  

# 지정한 값 불러오기
# checkpoint = torch.load('7emotions_all.tar')   
# model1.load_state_dict(checkpoint['model'])
# optimizer.load_state_dict(checkpoint['optimizer'])

In [17]:
# 앞에 클래스를 선언해줘야 모델 실행 가능

# kobert에서 vocab을 통해서 토큰화 진행
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
sources = ['공천포','휴애리 자연생활공원','성산일출봉(UNESCO 세계자연유산)','본태박물관','제주올레 14코스',
           '바이나흐튼 크리스마스 박물관','세계자동차 & 피아노박물관','한국야구명예전당(야구박물관)','초콜릿박물관']
# 테스트 함수 
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    # 평가 모드
    sentimentModel.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = sentimentModel(token_ids, valid_length, segment_ids)

        test_eval=[]

        # 학습위해 변환한 분류값 초기 분류값으로 변환
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            test_eval.append(sources[np.argmax(logits)])

        print(test_eval[0])

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [50]:
sentence = input("검색문장: ")
predict(sentence)

검색문장: 박물관 추천


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


초콜릿박물관
